In [1]:
from packages import *
import numpy as np
import tensorflow as tf
from tensorflow.data.experimental import AUTOTUNE

In [2]:
epsilon = 1e-7
fp = 'float32'
tf.config.gpu.set_per_process_memory_growth(True)
tf.config.gpu.set_per_process_memory_fraction(.05)
tf.keras.backend.set_floatx(fp)
tf.keras.backend.set_epsilon(epsilon)
tf.keras.backend.set_image_data_format('channels_last')

In [22]:
to_ix = True
if to_ix:
    all_key_to_ix = load_pickle(os.path.join(GLOVE_DIR, 'glove-byte-float32_to_ix.pkl'))
    j = [2]
    keys_to_ix = {}
    keys_to_ix[NULL_KEY.encode()] = 0
    keys_to_ix[UNK_KEY.encode()] = 1 
#ASSUMING ix 0 is for NULL and 1 for UNK

In [23]:
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[*value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[*value]))

In [24]:
def serialize_example(review, to_ix):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    
    
    
    feature = {
      'review': _int64_feature(review) if to_ix else _bytes_feature(review),
    }

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [25]:
chunk_count = 100

In [31]:
def _to_ix(review):
    ix = []
    for word in review:
        if word in keys_to_ix:
            ix.append(keys_to_ix[word])
        else:
            keys_to_ix[word] = j[0]
            ix.append(j[0])
            j[0]+=1
    return ix   

In [32]:
def _serialize(review, to_ix=False):
    if to_ix:
        review = _to_ix(review)
    return serialize_example(review, to_ix)

In [8]:
review_paths = [os.path.join(DATASET_DIR, 'preprocessed', 'npy', 'review-text-{:02d}.npy'.format(i)) for i in range(chunk_count)] #100 files

In [14]:
review = chunk

In [27]:
_serialize(review[12], to_ix=to_ix)

b'\n\xc2\x02\n\xbf\x02\n\x06review\x12\xb4\x02\x1a\xb1\x02\n\xae\x02\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x07\x11\x12\x13\x03\x14\x07\x15\x16\x17\x18\x19\x1a\x1b\x1c\x18\x1d\x1e\x1f !"#\x03$%&\x07\'\x10(\t)!\x17\x04*\t\x12\x03\x07+,-.\x03/&\x07012\x173*\x0345\x176\r7\x1789\x075\t!\x17\x18:;\x03\x0e<4*=\x10\x18>\x1e?\x034@A\x17#9\r\rBC\tDE\x07F\x034G\x1b5\tH-IJ\x03/K5\tLMNO$P\x178Q/\x17RS!\rTU!V$WXY"\x07#Z[\\Y]!^\x03/_`ab"cdef\tghY]\x18i\tjk=\x03\x18il\x03\x04f\x03\x07mnopq\r\r\r\x0erstu\x03v!\x18w\x03cdxyz\x07\x0c\x03\x18{\x1e|\x1b\x13}\x1b\n*H\t~[\x7f\x80\x01\r\x81\x01\x10!\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [30]:
_parse_function(_serialize(review[12], to_ix=to_ix), to_ix=True)

{'review': <tf.Tensor: id=4, shape=(300,), dtype=int64, numpy=
 array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,   7,  17,  18,  19,   3,  20,   7,  21,  22,  23,  24,
         25,  26,  27,  28,  24,  29,  30,  31,  32,  33,  34,  35,   3,
         36,  37,  38,   7,  39,  16,  40,   9,  41,  33,  23,   4,  42,
          9,  18,   3,   7,  43,  44,  45,  46,   3,  47,  38,   7,  48,
         49,  50,  23,  51,  42,   3,  52,  53,  23,  54,  13,  55,  23,
         56,  57,   7,  53,   9,  33,  23,  24,  58,  59,   3,  14,  60,
         52,  42,  61,  16,  24,  62,  30,  63,   3,  52,  64,  65,  23,
         35,  57,  13,  13,  66,  67,   9,  68,  69,   7,  70,   3,  52,
         71,  27,  53,   9,  72,  45,  73,  74,   3,  47,  75,  53,   9,
         76,  77,  78,  79,  36,  80,  23,  56,  81,  47,  23,  82,  83,
         33,  13,  84,  85,  33,  86,  36,  87,  88,  89,  34,   7,  35,
         90,  91,  92,  89,  93,  33,  94,   3,  47,  95,  96

In [9]:
for i in range(chunk_count):
    chunk = np.load(review_paths[i],allow_pickle=False)
    break
    chunk_tf = [_serialize(x, to_ix = to_ix) for x in chunk]
    chunk_ds = tf.data.Dataset.from_tensor_slices(chunk_tf)
    writer_path = os.path.join(DATASET_DIR, 'preprocessed', 'tfrecord', 'ix' if to_ix else 'xext',
                               'review-{}-{:02d}.tf'.format('ix' if to_ix else 'text', i))
    writer = tf.data.experimental.TFRecordWriter(writer_path)
    writer.write(chunk_ds)
    print(i+1, end=', ')
if to_ix:
    save_pickle(os.path.join(GLOVE_DIR, 'glove-byte-keys_to_ix.pkl'), keys_to_ix)

In [24]:
len(keys_to_ix)

170431

In [12]:
# for x in chunk_ds:break
# writer = tf.data.experimental.TFRecordWriter(writer_path)
# writer.write(chunk_ds)

In [28]:
def _parse_function(proto, to_ix):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'review': tf.io.FixedLenFeature([300,], tf.int64) if to_ix else tf.io.FixedLenFeature([300,], tf.string),}
    
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    return parsed_features 

In [15]:
dataset = tf.data.TFRecordDataset(writer_path)

In [16]:
dataset = dataset.map(lambda x: _parse_function(x, to_ix)['review'], num_parallel_calls=8)

In [17]:
for x in dataset:
    break